In [68]:
import dice

sample_specs = {
    "bs":3,
    "shots":"10",
    "s":6,
    "ap":1,
    "d":"2",
    "reroll_hit":"all", #"ones", "all", "none"
    "reroll_wound":"all", #"ones", "all", "none"
    "lightning_fast":True # True, False
}

In [2]:
defensive_specs = {
    "t":6,
    "save":4,
    "invul":5
}

In [3]:
def gen_results_dict(sides=6):
    results = dict()
    for x in range(1,sides+1):
        results[x]=0
    return results

In [4]:
def roll_x_dice(num, sides=6):
    if not isinstance(sides, int):
        raise Exception('Sides must be integer')
    if not isinstance(num, int):
        raise Exception('Num must be integer!')
    res = dice.roll(str(num)+'d'+str(sides))
    return res

In [5]:
def get_results(rolls, requirement, sides=6):
    successes = 0
    fails = 0
    results = gen_results_dict(sides)
    for result in rolls:
        results[result] += 1
        if result >= requirement:
            successes += 1
        else:
            fails += 1
    return {"successes": successes, "fails": fails, "individual_results": results}

In [6]:
res = roll_x_dice(100)
final=get_results(res, 4)


In [7]:
def reroll_results(original_results, requirement, sides=6):
    original_fails = 0
    final_successes = 0
    final_fails = 0
    results = gen_results_dict(sides)
    
    for result in original_results:
        if result >= requirement:
            final_successes += 1
            results[result] += 1
        else:
            original_fails +=1
            
    rerolls = roll_x_dice(original_fails, sides)
    for result in rerolls:
        results[result] += 1
        if result >= requirement:
            final_successes += 1
        else:
            final_fails += 1
    return {"successes": final_successes, "fails": final_fails, "individual_results": results}
    
    
    

In [8]:
def reroll_ones(original_results, requirement, sides=6):
    ones = 0
    final_successes = 0
    final_fails = 0
    results = gen_results_dict(sides)
    
    for result in original_results:
        if result >= requirement:
            final_successes += 1
            results[result] += 1
        elif result == 1:
            ones +=1
        else:
            results[result] += 1
            final_fails +=1
            
    rerolls = roll_x_dice(ones, sides)
    for result in rerolls:
        results[result] += 1
        if result >= requirement:
            final_successes += 1
        else:
            final_fails += 1
    return {"successes": final_successes, "fails": final_fails, "individual_results": results}

In [9]:
def parse_dice_string(dice_string):
    output = {
        "type_string":'',
        "number_of_dice":'1',
        "plus_modifier":0
    }
    d_index = dice_string.index('d')
    type_string = 'd'+dice_string[d_index+1]
    output["type_string"] = type_string
    if d_index != 0:
        number_of_dice = dice_string[0]
        output["number_of_dice"]=number_of_dice
    if "+" in dice_string:
        plus_index = dice_string.index('+')
        plus_modifier = dice_string[plus_index+1:]
        output["plus_modifier"]=int(plus_modifier)
    return output
        

In [10]:
parse_dice_string("3d3+3")

{'type_string': 'd3', 'number_of_dice': '3', 'plus_modifier': 3}

In [11]:
print(parse_dice_string('d6+2'))

{'type_string': 'd6', 'number_of_dice': '1', 'plus_modifier': 2}


In [12]:
def roll_dice_string(dice_string):
    if "d" in dice_string:
        parsed_dice_string = parse_dice_string(dice_string)
        dice_result = dice.roll(parsed_dice_string['number_of_dice']+parsed_dice_string['type_string'])
        return sum(dice_result)+ parsed_dice_string["plus_modifier"]
    else:
        return int(dice_string)

In [13]:
def determine_number_of_shots(specs):
    shots = specs["shots"]
    return roll_dice_string(shots)

In [14]:
def normalise_hit_requirement(req):
    if req>6:
        return 6
    elif req<2:
        return 2
    else:
        return req

def roll_to_hit(specs):
    number_of_shots = determine_number_of_shots(specs)
    hit_rolls = roll_x_dice(number_of_shots)
    hit_requirement = normalise_hit_requirement(specs["bs"]-1 if specs["lightning_fast"] else specs["bs"] )
    if specs["reroll_hit"]=="ones":
        return reroll_ones(hit_rolls, specs["bs"])
    elif specs["reroll_hit"]=="all":
        return reroll_results(hit_rolls, specs["bs"])
    else:
        return get_results(hit_rolls,specs["bs"] )

In [15]:
hit_results = roll_to_hit(sample_specs)

In [16]:
def get_wound_requirements(strength, toughness):
    if strength == toughness:
        return 4
    elif strength < toughness:
        if strength <= toughness/2:
            return 6
        else:
            return 5
    else :
        if strength >= toughness*2:
            return 2
        else:
            return 3
        

In [17]:
def roll_to_wound(specs, hit_results, defensive_specs):
    wound_requirement = get_wound_requirements(specs["s"], defensive_specs["t"])
    wound_rolls = roll_x_dice(hit_results['successes'])
    if specs["reroll_wound"]=="ones":
        return reroll_ones(wound_rolls, wound_requirement)
    elif specs["reroll_wound"]=="all":
        return reroll_results(wound_rolls,wound_requirement)
    else:
        return get_results(wound_rolls,wound_requirement)

In [18]:
wound_results = roll_to_wound(sample_specs,hit_results,defensive_specs)

In [19]:
def get_save_requirement(ap, save, invul=None):
    modified_save = save-ap
    if invul and modified_save > invul:
        return invul
    return modified_save

In [20]:
def roll_to_save(specs, wound_results, defensive_specs):
    save_requirement = get_save_requirement(specs["ap"], defensive_specs["save"], defensive_specs["invul"])
    save_rolls = roll_x_dice(wound_results['successes'])
    return get_results(save_rolls, save_requirement)

In [21]:
save_results = roll_to_save(sample_specs, wound_results,defensive_specs)

In [22]:
def get_damage_results(specs, save_results):
    total_damage = 0
    for wound in range(save_results["fails"]):
        total_damage += roll_dice_string(specs["d"])
    return total_damage        

In [23]:
damage_results = get_damage_results(sample_specs, save_results)

In [29]:
def simulate (offensive_specs, defensive_specs):
    hit_results = roll_to_hit(offensive_specs)
    wound_results = roll_to_wound(offensive_specs,hit_results,defensive_specs)
    save_results = roll_to_save(offensive_specs, wound_results,defensive_specs)
    damage_results = get_damage_results(offensive_specs, save_results)
    return damage_results

In [74]:
simulate(sample_specs, defensive_specs)

4